# Part 3: Weakly supervised part-of-speech tagging

In this part, we will work on a different type of task, which is called **sequence labeling**. Instead of having one label for an entire text, in sequence labeling, we assign a label to each token in the text.
Specifically we chose **Part-of-speech (POS) tagging**, where the goal is to assign a POS tag that indicates a grammatical type, to a word based on its definition and context.


<img src="../img/pos_tagging.png" width="800" style="display: block; margin: 0 auto" />



In order to perform weakly supervised POS tagging, we will employ the [skweak toolkit](https://github.com/NorskRegnesentral/skweak).
We will create labeling functions to assign POS tags based on _syntactic analysis_ and _grammatical rules_.


In [1]:
# Imports
%load_ext autoreload
%autoreload 2

import re
import os

import pandas as pd

import spacy
from spacy.tokens import Span
from spacy.training import Corpus

import skweak

from scripts.skweak_ner_eval import evaluate
from scripts.utils import load_data_split, get_frequent_words, tag_all

pd.set_option('display.max_rows', 500)

## POS tags

For this tutorial, we will use the following subset of the [universal POS tags](https://universaldependencies.org/u/pos/index.html):
1. **DET**: determiner, which is a word that modifies nouns or noun phrases and expresses the reference of the noun phrase in context.
2. **NUM**: numeral. It is a word that expresses a number and a relation to the number, such as quantity, sequence, frequency or fraction.
3. **PROPN**: proper noun is a noun that is the name of a specific individual, place, or object.
4. **ADJ**: adjective, which is a word that typically modifies nouns and specifies their properties or attributes.
5. **NOUN**: noun, which is a part of speech typically denoting a person, place, thing, animal or idea.

In [8]:
all_labels = ["DET", "NUM", "PROPN", "ADJ", "NOUN"]

## Load data

We will use the [English corpus](https://universaldependencies.org/treebanks/en_ewt/index.html) from Universal Dependencies, a framework that contains consistent grammatical annotations across many different languages.
The texts in the corpus come from five types of web media: weblogs, newsgroups, emails, reviews, and Yahoo! answers and consist of 254,825 words and 16,621 sentences.

Skweak operates on spaCy ``doc`` objects, so the dataset is loaded in this format.

In [23]:
# Load training data
train_docs = load_data_split("train", all_labels)

In [50]:
for doc in train_docs[:3]:
    skweak.utils.display_entities(doc)

## 3.1 Labeling functions

In the first step, we find the 200 most frequent words in our training corpus and use a lexicon to label these words. In the second step, we mannually annotate the 50 most frequent words.
Finally, for each POS tag we will create the following labeling functions: 

*   DET --> Lexicon with determiners.
*   NUM --> If the token is a number or a word indicating a number from 1 to 10.
*   PROPN --> A word that is capitalized.
*   ADJ --> List of prefixes and suffixes. Syntactic rules that check: 1. if the previous word is a form of "be" and 2. if the previous word is a determiner or numeral.
*   NOUN --> List of prefixes and suffixes. Syntactic rule checking if the previous word is a determiner, numeral or adjective.

#### Lexicon LF

In [29]:
# Get the 200 most frequent words in the training set
frequent_words = get_frequent_words(train_docs, 200)
print(frequent_words[:5])

["'s", "n't", 'would', 'one', 'like']


In [30]:
# Load the lexicon
with open("noun_vb_adj_list.txt") as f:
    lines = f.readlines()

# Create a dictionary with the words and their pos tags
lexicon = {}
for l in lines:
    values = l.replace("\n", "").split("\t")
    lexicon[values[0]] = values[1]

In [31]:
print("There are", len(lexicon), "words in the lexicon.")
print(list(lexicon.items())[:5])

There are 3387 words in the lexicon.
[('people', 'NOUN'), ('history', 'NOUN'), ('way', 'NOUN'), ('art', 'NOUN'), ('world', 'NOUN')]


In [32]:
# How many of the frequent words we found exist in the lexicon
len((list(set(frequent_words) & set(list(lexicon.keys())))))

121

In [43]:
# Lexicon LF
def frequent_word_detector(doc):
    for token in doc:
        # If the frequent word exists in the lexicon use its assigned pos tag
        if token.text.lower() in frequent_words and token.text.lower() in list(lexicon.keys()):
            yield token.i, token.i + 1, lexicon[token.text.lower()]


lexicon_lf = skweak.heuristics.FunctionAnnotator("frequent_words", frequent_word_detector)


#### Manual annotation LF

In [34]:
# Manual annotation
top50_words = get_frequent_words(train_docs, 50)
print(top50_words)

["'s", "n't", 'would', 'one', 'like', 'time', 'get', 'know', 'also', 'us', 'good', 'could', 'new', 'go', 'please', '$', 'people', 'may', 'back', 'said', 'even', 'work', 'bush', 'well', 'want', 'great', 'way', 'see', 'best', 'place', 'take', "'m", 'going', 'service', 'need', 'thanks', 'make', 'many', 'year', 'number', 'day', 'two', 'think', 'much', 'food', 'let', 'first', 'call', '2', 'help']


In [35]:
# Annotate the words that their POS tag exists in our chosen tag subset
manual_tags = {
    "one": "NUM",
    "like": "VERB",
    "time": "NOUN",
    "get": "VERB",
    "know": "VERB",
    "good": "ADJ",
    "could": "VERB",
    "new": "ADJ",
    "go": "VERB",
    "please": "VERB",
    "people": "NOUN",
    "said": "VERB",
    "work": "VERB",
    "bush": "NOUN",
    "want": "VERB",
    "great": "ADJ",
    "way": "NOUN",
    "see": "VERB",
    "best": "ADJ",
    "place": "NOUN",
    "take": "VERB",
    "going": "VERB",
    "service": "NOUN",
    "need": "VERB",
    "make": "VERB",
    "year": "NOUN",
    "number": "NOUN",
    "day": "NOUN",
    "two": "NUM",
    "think": "VERB",
    "food": "NOUN",
    "let": "VERB",
    "first": "ADJ",
    "call": "VERB",
    "2": "NUM",
    "help": "VERB"
}

In [36]:
# Manual POS tags LF
def manual_pos_tagger(doc):
    for token in doc:
        if token.text.lower() in manual_tags:
            yield token.i, token.i + 1, manual_tags[token.text.lower()]


manual_pos_lf = skweak.heuristics.FunctionAnnotator("manual_pos", manual_pos_tagger)


#### DET LF

In [37]:
# Use a lexicon of determiners
tries = skweak.gazetteers.extract_json_data("det.json")
det_lf = skweak.gazetteers.GazetteerAnnotator("determiners", tries, case_sensitive=False)


Extracting data from det.json
Populating trie for class DET (number: 47)


#### NUM LF

In [38]:
# Use a regular expression pattern to look for digits
def num_detector(doc):
    for token in doc:
        if re.search("\d+", token.text):
            yield token.i, token.i + 1, "NUM"

# Check if the token is the word of a number from 1 to 10
def num_word_detector(doc):
    for token in doc:
        if token.text.lower() in ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]:
            yield token.i, token.i + 1, "NUM"

num_lf1 = skweak.heuristics.FunctionAnnotator("numerals1", num_detector)
num_lf2 = skweak.heuristics.FunctionAnnotator("numerals2", num_word_detector)


#### PROPN LF

In [39]:
# Check if the fist letter of a word or the whole word is capitalized
def propn_detector(doc):
    for token in doc:
        if token.i == 0:
            # For the first word of a sentence, check if all letters are capitalized
            if token.text.isupper():
                yield token.i, token.i + 1, "PROPN"
        else:
            if token.text.isupper() or token.text[0].isupper():
                yield token.i, token.i + 1, "PROPN"


propn_lf = skweak.heuristics.FunctionAnnotator("proper_nouns", propn_detector)


#### ADJ LFs

In [45]:
# Look for common suffixes and prefixes
def adj_detector_suffixes(doc):
    suffixes = ("able", "al", "ful", "ic", "ive", "less", "ous", "y", "ish", "ible", "ent", "est")
    for token in doc:
        if len(token.text) > 3 and token.text.endswith(suffixes):
            yield token.i, token.i + 1, "ADJ"


# Look for common prefixes
def adj_detector_prefixes(doc):
    prefixes = ("un", "im", "in", "ir", "il", "non", "dis")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "ADJ"


# If the previous word is a form of "be" and the current word does not end with "ing" and was not labeled as DET, then it's an adjective
def adj_detector_synt1(doc):
    weak_labels = ["O"] * len(doc)
    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            prev = doc[token.i - 1].text.lower()
            if prev in ["be", "been", "being", "am", "is", "are", "was", "were"] and (
                    not token.text.endswith("ing")) and weak_labels[token.i] == "O":
                yield token.i, token.i + 1, "ADJ"


# If the previous word is labeld as DET or NUM, then the current word is an adjective
def adj_detector_synt2(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals2"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "ADJ"


adj_lf1 = skweak.heuristics.FunctionAnnotator("adjectives1", adj_detector_suffixes)
adj_lf2 = skweak.heuristics.FunctionAnnotator("adjectives2", adj_detector_prefixes)
adj_lf3 = skweak.heuristics.FunctionAnnotator("adjectives3", adj_detector_synt1)
adj_lf4 = skweak.heuristics.FunctionAnnotator("adjectives4", adj_detector_synt2)


#### NOUN LF

Let's create a labeling function that looks for common noun suffixes. Can you think of some?

In [41]:
# ***********************************
def noun_detector_suffixes(doc):
    suffixes = ("ment", "tion", "sion", "xion", "ant", "ent", "ee", "er", "or",
                "ism", "ist", "ness", "ship", "ity", "ance", "ence",
                "ar", "or", "y", "acy", "age")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().endswith(suffixes):
            yield token.i, token.i + 1, "NOUN"

# ***********************************

In [47]:
# Look for common prefixes
def noun_detector_prefixes(doc):
    prefixes = (
        "anti", "auto", "bi", "co", "counter", "dis", "ex", "hyper", "in", "inter", "kilo", "mal", "mega", "mis",
        "mini", "mono", "neo", "out", "poly", "pseudo", "re", "semi", "sub", "super", "sur", "tele", "tri", "ultra",
        "under", "vice")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "NOUN"


# # If the previous word is labeld as DET, NUM or ADJ, then the current word is an noun
def noun_detector_synt(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals2"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives2"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives3"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives4"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "NOUN"


noun_lf1 = skweak.heuristics.FunctionAnnotator("nouns1", noun_detector_suffixes)
noun_lf2 = skweak.heuristics.FunctionAnnotator("nouns2", noun_detector_prefixes)
noun_lf3 = skweak.heuristics.FunctionAnnotator("nouns3", noun_detector_synt)


## Apply LFs

In [48]:
# Put all LFs in a list
lfs = [
    lexicon_lf, manual_pos_lf, det_lf, 
    num_lf1, num_lf2, propn_lf,
    adj_lf1, adj_lf2, adj_lf3, adj_lf4,
    noun_lf1, noun_lf2, noun_lf3
]

train_docs = tag_all(train_docs, lfs)

In [49]:
# Print some of the assigned weak labels
for doc in train_docs[:3]:
    skweak.utils.display_entities(doc, ["determiners", "nouns1"])

In [51]:
# Train HMM
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm = hmm.fit(train_docs, n_iter=8)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 2


         1     -518827.3307             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 3


         2     -490363.7763      +28463.5544


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 4


...

         6     -462378.4837       +3688.3311


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 8


         7     -457974.1572       +4404.3265


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents


         8     -451644.7334       +6329.4238


In [52]:
# Majority voting
mv = skweak.aggregation.MajorityVoter("mv", all_labels)

In [53]:
# Apply LFs, HMM and MV to the test docs
test_docs = load_data_split("test", all_labels)
test_docs = tag_all(test_docs, lfs + [mv, hmm])

## Evaluate

#### Which POS tags are easier to detect?

* We see that POS tags like determiners and numerals are easier to detect and we can achieve a good F1 score with just one or two simple LFs.


In [62]:
df = evaluate(test_docs, all_labels, [
    "determiners", "numerals1", "numerals2", "proper_nouns"
])

In [72]:
df.loc[["DET", "NUM", "PROPN"]]

tok_precision  tok_recall  tok_f1
label proportion model                                          
DET   18.4 %     determiners           0.678       0.998   0.808
                 numerals1             0.000       0.000   0.000
                 numerals2             0.000       0.000   0.000
                 proper_nouns          0.000       0.000   0.000
NUM   5.2 %      determiners           0.000       0.000   0.000
                 numerals1             0.813       0.802   0.808
                 numerals2             0.786       0.144   0.244
                 proper_nouns          0.000       0.000   0.000
PROPN 20.1 %     determiners           0.000       0.000   0.000
                 numerals1             0.000       0.000   0.000
                 numerals2             0.000       0.000   0.000
                 proper_nouns          0.568       0.782   0.658

* Other POS tags like adjectives and nouns, which rely more on the context are harder to detect and require more complicated rules.

In [73]:
df = evaluate(test_docs, all_labels, [
    "adjectives1", "adjectives2", "adjectives3", "adjectives4",
    "nouns1", "nouns2", "nouns3"
])

In [74]:
df.loc[["ADJ", "NOUN"]]

tok_precision  tok_recall  tok_f1
label proportion model                                         
ADJ   16.4 %     adjectives1          0.336       0.339   0.338
                 adjectives2          0.254       0.049   0.082
                 adjectives3          0.297       0.091   0.140
                 adjectives4          0.176       0.319   0.226
                 nouns1               0.000       0.000   0.000
                 nouns2               0.000       0.000   0.000
                 nouns3               0.000       0.000   0.000
NOUN  40.0 %     adjectives1          0.000       0.000   0.000
                 adjectives2          0.000       0.000   0.000
                 adjectives3          0.000       0.000   0.000
                 adjectives4          0.000       0.000   0.000
                 nouns1               0.397       0.214   0.278
                 nouns2               0.347       0.099   0.154
                 nouns3               0.362       0.603   0.452

#### Which type of LF works the best?

* For adjectives the LF that uses suffixes works the best, while the syntactic rules are less accurate. On the contrary, for nouns the LF that is based on syntactic analysis has the best results. For both POS tags, the LFs that use prefixes do not yield good results.

#### Which aggregator works best?

* Despite its simplicity, majority voting outperforms HMM on almost all of the POS tags and overall achieves a higher macro F1 score.

In [75]:
df = evaluate(test_docs, all_labels, ["mv", "hmm"])

In [76]:
df

tok_precision  tok_recall  tok_f1
label    proportion model                                   
ADJ      16.4 %     hmm            0.189       0.335   0.242
                    mv             0.425       0.455   0.440
DET      18.4 %     hmm            0.688       0.935   0.792
                    mv             0.710       0.835   0.768
NOUN     40.0 %     hmm            0.333       0.195   0.246
                    mv             0.387       0.440   0.412
NUM      5.2 %      hmm            0.622       0.157   0.250
                    mv             0.866       0.724   0.788
PROPN    20.1 %     hmm            0.610       0.258   0.362
                    mv             0.579       0.462   0.514
macro               hmm            0.488       0.376   0.424
                    mv             0.593       0.583   0.588
micro               hmm            0.418       0.365   0.390
                    mv             0.509       0.534   0.522
weighted            hmm            0.445       0.365   0.402
                    mv             0.516       0.534   0.524

## Run LFs on a smaller subset

What if we use a smaller amount of training data? How will that affect the performance?

In [139]:
subset_train_docs = load_data_split("train", all_labels, 500)

#### Annotate the most common 50 words

In [140]:
# Manual annotation
top50_words = get_frequent_words(subset_train_docs, 50)
print(top50_words)

["'s", 'bush', 'al', 'india', 'would', 'iraq', 'us', 'iraqi', "n't", 'one', 'many', 'even', 'indian', 'said', 'new', 'war', 'musharraf', 'peace', 'years', 'country', 'military', 'israel', 'two', 'also', 'national', 'time', 'chernobyl', 'pakistan', 'government', 'kashmir', 'sri', 'elections', 'know', 'qaeda', 'may', 'president', 'power', 'last', 'another', 'lanka', 'posada', 'back', 'could', 'state', 'general', 'made', 'much', 'party', 'united', 'people']


In [141]:
manual_tags = {
    "bush": "NOUN",
    "al": "PROPN",
    "india": "PROPN",
    "iraq": "PROPN",
    "iraqi": "ADJ",
    "indian": "ADJ",
    "said": "VERB",
    "new": "ADJ",
    "war": "NOUN",
    "musharraf": "PROPN",
    "peace": "NOUN",
    "years": "NOUN",
    "country": "NOUN",
    "military": "NOUN",
    "israel": "PROPN",
    "two": "NUM",
    "national": "ADJ",
    "time": "NOUN",
    "chernobyl": "PROPN",
    "pakistan": "PROPN",
    "government": "NOUN",
    "kashmir": "PROPN",
    "sri": "PROPN",
    "elections": "NOUN",
    "know": "VERB",
    "qaeda": "PROPN",
    "president": "NOUN",
    "power": "NOUN",
    "last": "NOUN",
    "another": "ADJ",
    "lanka": "PROPN",
    "posada": "PROPN",
    "could": "VERB",
    "general": "ADJ",
    "made": "VERB",
    "party": "NOUN",
    "united": "VERB",
    "people": "NOUN",
}

In [142]:
# ***********************************

# Apply LFs to the subset docs
for doc in subset_train_docs:
    for lf in lfs:
        doc = lf(doc)

In [143]:
# Train HMM
hmm_2 = skweak.aggregation.HMM("hmm_2", all_labels, redundancy_factor=1.0)
hmm_2 = hmm_2.fit(subset_train_docs, n_iter=8)

Starting iteration 1
Finished E-step with 500 documents
Starting iteration 2


         1      -33070.3558             +nan


Finished E-step with 500 documents
Starting iteration 3


         2      -31391.1937       +1679.1620


Finished E-step with 500 documents
Starting iteration 4


...

         6      -28811.0492        +476.4665


Finished E-step with 500 documents
Starting iteration 8


         7      -28586.8415        +224.2078


Finished E-step with 500 documents


         8      -28536.3379         +50.5036


In [144]:
# Majority voting
mv = skweak.aggregation.MajorityVoter("mv", all_labels)


In [145]:
# Apply LFs, HMM and MV to the test docs

test_docs_2 = load_data_split("test", all_labels)

for doc in test_docs_2:
    for lf in  lfs + [mv, hmm_2]:
        doc = lf(doc)

In [146]:
# Evaluate
subset_df = evaluate(test_docs_2, all_labels, [
    # "common_words", "manual_pos",
    # "determiners", "numerals", "proper_nouns",
    # "adjectives1", "adjectives2", "adjectives3", "adjectives4",
    # "nouns1", "nouns2", "nouns3", 
    "mv", "hmm_2"
])

In [147]:
subset_df

tok_precision  tok_recall  tok_f1
label    proportion model                                   
ADJ      17.3 %     hmm_2          0.185       0.331   0.238
                    mv             0.367       0.389   0.378
DET      19.4 %     hmm_2          0.688       0.963   0.802
                    mv             0.708       0.845   0.770
NOUN     42.2 %     hmm_2          0.321       0.173   0.224
                    mv             0.346       0.386   0.364
PROPN    21.2 %     hmm_2          0.600       0.581   0.590
                    mv             0.576       0.482   0.524
macro               hmm_2          0.448       0.512   0.478
                    mv             0.499       0.526   0.512
micro               hmm_2          0.435       0.439   0.436
                    mv             0.467       0.496   0.482
weighted            hmm_2          0.428       0.440   0.434
                    mv             0.468       0.496   0.482

In [148]:
df

tok_precision  tok_recall  tok_f1
label    proportion model                                   
ADJ      17.3 %     hmm            0.183       0.323   0.234
                    mv             0.367       0.389   0.378
DET      19.4 %     hmm            0.687       0.912   0.784
                    mv             0.708       0.845   0.770
NOUN     42.2 %     hmm            0.299       0.176   0.222
                    mv             0.346       0.386   0.364
PROPN    21.2 %     hmm            0.623       0.279   0.386
                    mv             0.576       0.482   0.524
macro               hmm            0.448       0.422   0.434
                    mv             0.499       0.526   0.512
micro               hmm            0.404       0.366   0.384
                    mv             0.467       0.496   0.482
weighted            hmm            0.423       0.366   0.392
                    mv             0.468       0.496   0.482